In [ ]:
### to perform double segmentation, a standard and an ROI model is needed
### we will use the previously trained standard deeplab example and ROI deeplab example

In [ ]:
import torch

import general_config
import constants
import numpy as np
import save_complete_nii_results
from utils import inference
from utils import prepare_models
from utils.training_utils.box_utils import convert_offsets_to_bboxes, wh2corners
from utils.training_utils import training_processing
from utils.ROI_crop import roi_crop
from utils.ROI_crop import predict_ROI
from utils.training_utils import prints, training_setup
from utils import metrics
from utils import visualization


def get_double_seg_predictions(model_a, model_b, dataset, logsoft_preds=False):
    """
    Args:
    model_a: str -> model to extract ROI (standard)
    model_b: str -> model to predict from ROI
    dataset: str
    """
    models, params_list, configs_list = prepare_models.prepare([model_a, model_b], dataset)
    model_a, model_b = models
    validation_loader = training_setup.prepare_val_loader(params_list[0], configs_list[0])
    validation_loader_2 = training_setup.prepare_val_loader(params_list[1], configs_list[1])
    
    coords_n_scores = predict_ROI.get_segmentor_pred_coords(validation_loader, model_a, params_list[0], configs_list[0])
    _, predictions_double_seg = predict_ROI.segment_with_computed_ROI(validation_loader_2, model_b, params_list[1], configs_list[1], coords_n_scores, logsoft_preds=logsoft_preds)
    
    return predictions_double_seg, validation_loader_2

In [ ]:
# perform double segmentation, add err_margin to standard model params.json
_, _ = get_double_seg_predictions("Deeplab_standard_example", "Deeplab_ROI_example", "ACDC_training")

In [ ]:
### Ensembling: softmax predictions of different models are summed and averaged

In [ ]:
# ensembling two standard models

In [ ]:
# load the model weigths, again using the previously trained example models
models, params_list, configs_list = prepare_models.prepare(["Deeplab_standard_example", "2D_Unet_standard_example"], "ACDC_training")
model_a, model_b = models
validation_loader_a = training_setup.prepare_val_loader(params_list[0], configs_list[0])
validation_loader_b = training_setup.prepare_val_loader(params_list[1], configs_list[1])

In [ ]:
# perform inference with both
predictions_deeplab, _ = inference.run_model_inference(validation_loader_a, model_a, params_list[0], logsoft_preds=True)
predictions_unet, _ = inference.run_model_inference(validation_loader_b, model_b, params_list[1], logsoft_preds=True)

In [ ]:
# finally, ensemble standard models
logsoftmax_volumes = [predictions_unet, predictions_deeplab]
combined_volumes = inference.ensemble_inference(logsoftmax_volumes, validation_loader_a)

In [ ]:
# it is also possible to ensemble standard and ROI models
# add logsoft flag
double_seg_preds, _ = get_double_seg_predictions("Deeplab_standard_example", "Deeplab_ROI_example", "ACDC_training", logsoft_preds=True)

In [ ]:
# ensemble standard models and double segmentation output
logsoftmax_volumes = [predictions_unet, predictions_deeplab, double_seg_preds]
combined_volumes = inference.ensemble_inference(logsoftmax_volumes, validation_loader_a)

In [ ]:
### TESTING

In [ ]:
def get_double_seg_predictions_test(model_a, model_b, dataset):
    """
    Args:
    model_a: str -> model to extract ROI
    model_b: str -> model to predict from ROI
    dataset: str
    """
    coords_n_scores = get_test_coords(model_a, dataset)
    model_roi, params, config = prepare_models.prepare([model_b], dataset)
    test_loader = training_setup.prepare_test_loader(params, config)
    
    predictions_double_seg = predict_ROI.segment_with_computed_ROI_test(test_loader, model_roi, params, config, coords_n_scores, logsoft_preds=True)
    
    return predictions_double_seg

def get_test_coords(model, dataset):
    model, params, config = prepare_models.prepare([model], dataset)
    test_loader = training_setup.prepare_test_loader(params, config)
    coords_n_scores = predict_ROI.get_segmentor_pred_coords(test_loader, model, params, config)
    return coords_n_scores

In [ ]:
# the test set is in a directory of its own, so copy/move the previously trained models' experiments folders
# in experiments/acdc_test_set
# also, don't forget to change the dataset name from ACDC_training to acdc_test_set in each config.json

In [ ]:
# running inference and ensemble on the test set is very similar to before

In [ ]:
models, params_list, configs_list = prepare_models.prepare(["Deeplab_standard_example", "2D_Unet_standard_example"], "acdc_test_set")
model_a, model_b = models
test_loader = training_setup.prepare_test_loader(params_list[0], configs_list[0])
test_loader_2 = training_setup.prepare_test_loader(params_list[1], configs_list[1])

In [ ]:
predictions_deeplab = inference.run_model_test_inference(test_loader, model_a, params_list[0], logsoft_preds=True)
predictions_unet = inference.run_model_test_inference(test_loader_2, model_b, params_list[1], logsoft_preds=True)

In [ ]:
predictions_double = get_double_seg_predictions_test("Deeplab_standard_example", "Deeplab_ROI_example", "acdc_test_set")

In [ ]:
logsoftmax_volumes = [predictions_unet, predictions_deeplab, predictions_double]
combined_volumes = inference.ensemble_inference_test(logsoftmax_volumes)

In [ ]:
# saving the results to a folder
save_complete_nii_results.save_predictions_test(combined_volumes, "ResNeXt_DeepLabV3_plus", "acdc_test_set")